### The notbook shows results for MLR, and XGB for table 3. However the code can be changed to produce other table resluts.

## Importing packges

In [3]:
from PIL import Image, ImageSequence
import glob
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV,TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
#######################################
import time
from sklearn.metrics import *
from skimage.util.shape import view_as_windows


## Importaing the data

In [4]:
# we are reading the same file for the input and the output
yFiles = sorted(glob.glob ('C:/Users/Admin/Documents/Work/2020-Masters/data/proj1-imgPred/water/data/NewIn3/*.png')) # Change Cell
xFiles = sorted(glob.glob ('C:/Users/Admin/Documents/Work/2020-Masters/data/proj1-imgPred/water/data/NewIn3/*.png')) # Change Cell

In [5]:

# the following function makes a sequence of images in the fromat of (102, 72, 928)
 ## 102 is the number of totall sequences, each sequence has 72 arrays, each array has 928 pixels
    
def makingDataset (xPathArr, yPathArr,seqLen, numDayAhead):
    DataX = []
    DataY = []
    
    w,h = Image.open(xPathArr[0]).size
    totPixels = w*h
    
    for i in range(len(xPathArr)):
        try:
            nameY = os.path.basename(yPathArr[i+seqLen+numDayAhead]).split(".")
#             print(nameY[0][-1])
            if (nameY[0][-1] != "y"):
                DataX1 = []
                imgX0 = Image.open(xPathArr[i]).convert("L")
                imgXSeq_Numpy = np.array(imgX0)
                DataX1.append (imgXSeq_Numpy)  
                for j in range(seqLen-1):

                    imgX = Image.open(xPathArr[i+j+1]).convert("L")  # to open the image
                    imgX_Numpy = np.array(imgX)#to get the data f the next image in the sequence
                    DataX1.append (imgX_Numpy)

                imgY = Image.open(yPathArr[i+seqLen+numDayAhead]).convert("L")

                DataX.append(( DataX1 )) # append the features n for instance i into a list, this
#                 print(nameY[0][-2:])
                DataY.append([np.array(imgY), int(nameY[0][-2:])] )
                     
        except: # means out of range
            
            return DataX, DataY
        
    return DataX, DataY



# The following function takes the above tensor (array) fromat into a normal data sets with  (54320, 73)
    ## where the 54320 stands for the pixels, and 73 stands for the features, (72 pixels, and plus 1 feat = pixel index
def makingDataset2 (dataX, dataY,feat):
    window_shape = (5,5)
    pad =2

    Data = []
    DataX1 = []
    DataY1 = []
    pixInd_X = []
    pixInd_Y = []
    
    for seq in range (len(dataX)):
        for pixel_i in range (len(dataX[0][0])):
            for pixel_j in range (len(dataX[0][0][0])):
                Data = []
                if (dataY[seq][0][pixel_i][pixel_j] != 0):

                    for time in feat:
                        Data.append(np.sqrt(dataX[seq][time][pixel_i][pixel_j]))
                        
                    Data = np.append(Data, [pixel_i , pixel_j, dataY[seq][1]]) # pixel position (i, j), and time stamp
                    pixInd_X.append(pixel_i)
                    pixInd_Y.append(pixel_j)
    
                    DataX1.append(Data)
                    DataY1.append(dataY[seq][0][pixel_i][pixel_j])
                    
    return DataX1, DataY1,pixInd_X, pixInd_Y
        
        
    


In [6]:
seqLengthArr = [12]
monthAhead = [ 0]# 0 stands for 1 month ahead
features = [[0, 11], [0, 11, 1], [0, 11, 1 ,10], [0, 11, 1 ,2], [0, 11, 1, 10, 2, 9], [0, 11, 1, 10, 2, 9, 3], [0, 11, 1, 10, 2, 9, 3, 8, 4]]

# estimator = [LinearRegression() ]

# estimator = [LinearRegression(), MLPRegressor(),XGBRegressor(n_jobs=-1), RandomForestRegressor(n_jobs=-1), SVR() ]
estimator = [ XGBRegressor(), LinearRegression() ]

In [7]:
for lag in seqLengthArr:
    for month in monthAhead:
        for feat in features:
#             for est in estimator:

            DataX, DataY = makingDataset(xFiles, yFiles, lag, month)

            DataX = np.array(DataX)
            DataY = np.array(DataY)

            print(DataX.shape)
            print(DataY.shape)
            DataXX, DataYY, pixelPOSX, pixelPOSY = makingDataset2(DataX, DataY, feat) 

            DataXX = np.float64(np.array(DataXX))# 
            DataYY = np.sqrt(np.float64(np.array(DataYY)))#

        ################################## splitting the Data #####################################
            XtestImg = DataXX[ -(1774*12) : ] # testing of 12 images
            YtestImg = DataYY[  -(1774*12) : ]
            indexImgX = pixelPOSX[  -(1774*12) : ]
            indexImgY = pixelPOSY[  -(1774*12) : ]

            Xtrain = DataXX[0: -(1774*12)  ]
            Ytrain = DataYY[0: -(1774*12)  ]
        ###################################################################
            print(DataXX.shape)
            print(DataYY.shape)
            print(Xtrain.shape)
            print(Ytrain.shape)
            print(XtestImg.shape)
        ################################################################################
            for est in estimator:
                print()
#                 print("Heloo every one,this is "+str(est)+"on ("+str(feat)+", x, y, TS) features: " +str(month)
#                       +" month ahead")
                print("Heloo every one,this is "+str(est)+"on ("+str(feat)+") features: " +str(month)
                      +" month ahead")
                print()

                reg =  est
#                 print(reg)
                reg.fit(Xtrain, Ytrain)
                y_predImg = reg.predict(XtestImg)

            ###################################################################################


            ##################################################################################

                mae = mean_absolute_error(YtestImg**2, y_predImg**2)
                mse = mean_squared_error(YtestImg**2, y_predImg**2)
                rmse = np.sqrt(mean_squared_error(YtestImg**2, y_predImg**2))
                r2 = r2_score(YtestImg**2, y_predImg**2)
                print()
                print("Testing Set: ")
                print("MAE: " + str(mae))
                print("MSE: " + str(mse))
                print("RMSE: " + str(rmse))
                print("r2score: " + str(r2))
                print()    
                print("Image")
#                 predictedImg = (y_predImg[ -1774 : ]**2)*10
#                 originalImg = (YtestImg [ -1774 : ]**2)*10
#                 indexImgX = indexImgX[ -1774 : ]
#                 indexImgY = indexImgY[ -1774 : ]

#                 predImg = convToImg (predictedImg, indexImgX, indexImgY)
#                 origImg = convToImg (originalImg, indexImgX, indexImgY)
#                 print("predicted Image")
#                 plt.imshow(predImg, cmap='gray', vmin=0, vmax=255)
#                 plt.show()
#                 print("Original Image")
#                 plt.imshow(origImg, cmap='gray', vmin=0, vmax=255)
#                 plt.show()
                    

(161, 12, 51, 47)
(161, 2)
(285614, 5)
(285614,)
(264326, 5)
(264326,)
(21288, 5)

Heloo every one,this is XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)on ([0, 11]) features: 0 month ahead


Testing Set: 
MAE: 2.3429696736076817
MSE: 30.74460462046244
RMSE: 5.544781746873581
r2score: 0.4965463657229826

Image

Heloo every one,this is LinearRegression()on ([0, 11]) features: 0 month ahead


Testing Set: 
MAE: 2.857449484